# Descripción:

Visualización interactiva de descriptores 

## Recuperación por PICKLE

In [2]:
import sys
from os import path, mkdir, listdir, chdir
import pickle
import cv2 as cv
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import matplotlib.pyplot as plt
from skimage import io

modulos_path = path.abspath('../minIA')
if modulos_path not in sys.path:
    sys.path.append(modulos_path)
    
from utiles import lectura_img


In [3]:
#Recibe lista de keypoints de una imagen, crea los objetos KP de openCV
def genKeyPoints( kp_img ):
    keypoints = list()
    for kp in kp_img:
        keypoints.append(cv.KeyPoint(kp[0], kp[1], kp[2]))
    return keypoints

#### Rutas de ejecución

In [4]:
@interact_manual
def load_images(pickle_path='', image_dir='', 
                image_dir_out=''):
    pickle_file = open(path.abspath(pickle_path), 'rb')
    
    args = pickle.load(pickle_file)
    desc_imgs = pickle.load(pickle_file)

    indexes = range(len(desc_imgs))
    for i in indexes:
        imagen = path.abspath(image_dir + desc_imgs[i]['name_img'])
        img = cv.imread(imagen)
        gray= cv.cvtColor(img,cv.COLOR_BGR2GRAY)
        keypoints = genKeyPoints(desc_imgs[i]['keypoints'])
        img=cv.drawKeypoints(gray,keypoints,img,flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        cv.imwrite(image_dir_out+desc_imgs[i]['name_img'],img)
    
    global dir_out
    dir_out=image_dir_out
    print('Listo!' )

interactive(children=(Text(value='', description='pickle_path'), Text(value='', description='image_dir'), Text…

In [5]:
@interact
def show_images(img=listdir(dir_out)):
    chdir(dir_out)
    image=io.imread(path.abspath(img))
    plt.imshow(image)

NameError: name 'dir_out' is not defined

## Visualización tópicos

In [25]:
import pickle
import pandas as pd
import numpy as np

In [13]:
topics_path = '/media/working/minia/minhash/full_dataset_CentxImg.models'
topics = open(topics_path, 'r')

In [14]:
#Lee el primer tópico
topic = topics.readline()

In [93]:
# Obtiene los centroides del primer tópico
centroides = topic.split()[1:]
centroides = [ int(cent.split(':')[0]) for cent in centroides]


In [29]:
pickle_file = open('/media/working/minia/descriptores/centroidesXimagen.pickle','rb')
images_descr = pickle.load(pickle_file )
pickle_file.close()

In [107]:
def indicesTopicos (centroides, images_descr):
    tam_cents = len(centroides)
    img_index = 0
    images = list()
    for descrp in images_descr[0]:
        tam_descp = len(descrp)
        
        #Overlaping
        max_ = round( min(tam_descp, tam_cents)*.30 ) #Requiere un 30%
        posc_index = 0
        posc = list()
        for cent in descrp:
            if cent in centroides:
                posc.append(posc_index)
            posc_index += 1
        if len(posc) > max_ : #No puede ser >= porque max_ puede ser cero
            images.append((img_index,posc)) #Añade una tupla, del indice de la imagen y los indices de los KP
        img_index += 1
    return images

In [108]:
lista = indicesTopicos (centroides, images_descr[0:100])
print(lista[0])

(21, [173, 176, 191, 220, 227, 251, 262, 362, 380, 445, 457, 459, 469, 472, 539, 563, 572, 574, 578, 585, 592])
